# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 9:29AM,256682,10,Clear-9123589,"ClearSpec, LLC",Released
1,Feb 6 2023 9:21AM,256681,16,Em-BenHotel20623,"Emersa Waterbox, LLC",Released
2,Feb 6 2023 9:21AM,256681,16,Em-Edition20623,"Emersa Waterbox, LLC",Released
3,Feb 6 2023 9:21AM,256681,16,Em-PalmBeach20623,"Emersa Waterbox, LLC",Released
4,Feb 6 2023 9:21AM,256681,16,Em-MrCHotel20623,"Emersa Waterbox, LLC",Released
5,Feb 6 2023 9:21AM,256681,16,Em-ASH20623,"Emersa Waterbox, LLC",Released
6,Feb 6 2023 9:12AM,256679,10,SNAP34596297,"Snap Medical Industries, LLC",Released
7,Feb 6 2023 9:12AM,256679,10,SNAPW603379429,"Snap Medical Industries, LLC",Released
8,Feb 6 2023 9:12AM,256679,10,9122505,"Snap Medical Industries, LLC",Released
9,Feb 6 2023 9:12AM,256680,10,Eme-108459,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,256678,Released,1
19,256679,Released,3
20,256680,Released,2
21,256681,Released,5
22,256682,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256678,NaN,1.0
256679,NaN,3.0
256680,NaN,2.0
256681,NaN,5.0
256682,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256651,1.0,0.0
256655,2.0,0.0
256656,0.0,3.0
256659,1.0,0.0
256662,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256651,1,0
256655,2,0
256656,0,3
256659,1,0
256662,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256651,1,0
1,256655,2,0
2,256656,0,3
3,256659,1,0
4,256662,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256651,1,
1,256655,2,
2,256656,,3
3,256659,1,
4,256662,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC"
1,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC"
6,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC"
9,Feb 6 2023 9:12AM,256680,10,Emerginnova
11,Feb 6 2023 9:07AM,256678,10,"Citieffe, Inc."
12,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation
14,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation
45,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation
97,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation
133,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",,1
1,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC",,5
2,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC",,3
3,Feb 6 2023 9:12AM,256680,10,Emerginnova,,2
4,Feb 6 2023 9:07AM,256678,10,"Citieffe, Inc.",,1
5,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,,2
6,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,,31
7,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation,,52
8,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation,,36
9,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation,,48


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1,
1,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC",5,
2,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC",3,
3,Feb 6 2023 9:12AM,256680,10,Emerginnova,2,
4,Feb 6 2023 9:07AM,256678,10,"Citieffe, Inc.",1,
5,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,2,
6,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,31,
7,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation,52,
8,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation,36,
9,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation,48,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1,
1,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC",5,
2,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC",3,
3,Feb 6 2023 9:12AM,256680,10,Emerginnova,2,
4,Feb 6 2023 9:07AM,256678,10,"Citieffe, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1.0,NaN
1,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC",5.0,NaN
2,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC",3.0,NaN
3,Feb 6 2023 9:12AM,256680,10,Emerginnova,2.0,NaN
4,Feb 6 2023 9:07AM,256678,10,"Citieffe, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1.0,0.0
1,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC",5.0,0.0
2,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC",3.0,0.0
3,Feb 6 2023 9:12AM,256680,10,Emerginnova,2.0,0.0
4,Feb 6 2023 9:07AM,256678,10,"Citieffe, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3593427,264.0,0.0
12,256663,10.0,47.0
16,513343,6.0,0.0
19,1026646,5.0,3.0
22,256651,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3593427,264.0,0.0
1,12,256663,10.0,47.0
2,16,513343,6.0,0.0
3,19,1026646,5.0,3.0
4,22,256651,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,264.0,0.0
1,12,10.0,47.0
2,16,6.0,0.0
3,19,5.0,3.0
4,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,264.0
1,12,Released,10.0
2,16,Released,6.0
3,19,Released,5.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,22
Status,,,,,
Executing,0.0,47.0,0.0,3.0,1.0
Released,264.0,10.0,6.0,5.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,22
0,Executing,0.0,47.0,0.0,3.0,1.0
1,Released,264.0,10.0,6.0,5.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,22
0,Executing,0.0,47.0,0.0,3.0,1.0
1,Released,264.0,10.0,6.0,5.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()